<a href="https://colab.research.google.com/github/muhammadsajad/Automatic_lesion_detection_using_periapical_xrays/blob/main/Method_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Method A**


1.   Feature extraction using vgg16




In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
import random
from shutil import copyfile


from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
# from keras.layers.normalization import BatchNormalization
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16

In [2]:
# Give the path of kaggle.jason file in order to download data directly from kaggle.
import os
os.environ['KAGGLE_CONFIG_DIR']='/content'

In [3]:
# This is the kaggle api of data set.
!kaggle datasets download -d muhammadsajad/periapical-xrays

 87% 134M/153M [00:00<00:00, 228MB/s]
100% 153M/153M [00:00<00:00, 233MB/s]


In [4]:
# Unzip the data set
!unzip \*.zip && rm *.zip

Archive:  periapical-xrays.zip
  inflating: Periapical_Xrays/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (10).JPG  
  inflating: Periapical_Xrays/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (100).JPG  
  inflating: Periapical_Xrays/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (101).JPG  
  inflating: Periapical_Xrays/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (102).JPG  
  inflating: Periapical_Xrays/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (103).JPG  
  inflating: Periapical_Xrays/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (104).JPG  
  inflating: Periapical_Xrays/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (105).JPG  
  inflating: Periapical_Xrays/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (106).JPG  
  inflating: Periapical_Xrays/Primary Endo with Secondary Perio/Primary Endo with Secondar

In [5]:
# Display total number of images present in each category
source_path = '/content/Periapical_Xrays'

source_path_Primary_Endo_with_Secondary_Perio = os.path.join(source_path, 'Primary Endo with Secondary Perio')
source_path_Primary_Endodontic_Lesion = os.path.join(source_path, 'Primary Endodontic Lesion')
source_path_Primary_Perio_with_Secondary_Endo = os.path.join(source_path, 'Primary Perio with Secondary Endo')
source_path_Primary_Periodontal_Lesion = os.path.join(source_path, 'Primary Periodontal Lesion')
source_path_True_Combined_Lesions = os.path.join(source_path, 'True Combined Lesions')

# # Deletes all non-image files (there are two .db files bundled into the dataset)
# !find /tmp/PetImages/ -type f ! -name "*.jpg" -exec rm {} +

# os.listdir returns a list containing all files under the given path
print(f"There are {len(os.listdir(source_path_Primary_Endo_with_Secondary_Perio))} images of Primary Endo with Secondary Perio.")
print(f"There are {len(os.listdir(source_path_Primary_Endodontic_Lesion))} images of Primary Endodontic Lesion.")
print(f"There are {len(os.listdir(source_path_Primary_Perio_with_Secondary_Endo))} images of Primary Perio with Secondary Endo.")
print(f"There are {len(os.listdir(source_path_Primary_Periodontal_Lesion))} images of Primary Periodontal Lesion.")
print(f"There are {len(os.listdir(source_path_True_Combined_Lesions))} images of True Combined Lesions.")

There are 122 images of Primary Endo with Secondary Perio.
There are 124 images of Primary Endodontic Lesion.
There are 39 images of Primary Perio with Secondary Endo.
There are 118 images of Primary Periodontal Lesion.
There are 131 images of True Combined Lesions.




Now we create two trianing and validation directories and in that directories creating subdirectories for each catogory/class.

In [6]:
import shutil
# Define root directory
root_dir = '/content/Xrays'

# Empty directory to prevent FileExistsError is the function is run several times
if os.path.exists(root_dir):
  shutil.rmtree(root_dir)

# create_train_val_dirs
def create_train_val_dirs(root_path):
  """
  Creates directories for the train and test sets

  Args:
    root_path (string) - the base directory path to create subdirectories from


  """



  #This for creating training subdirectory
  parent_dir = root_path + "/training"
  path=os.path.join(parent_dir,'Primary Endo with Secondary Perio')
  os.makedirs(path)
  path=os.path.join(parent_dir,'Primary Endodontic Lesion')
  os.makedirs(path)
  path=os.path.join(parent_dir,'Primary Perio with Secondary Endo')
  os.makedirs(path)
  path=os.path.join(parent_dir,'Primary Periodontal Lesion')
  os.makedirs(path)
  path=os.path.join(parent_dir,'True Combined Lesions')
  os.makedirs(path)

  # This for creating validation subdirectory
  parent_dir=root_path + "/validation"
  path=os.path.join(parent_dir,'Primary Endo with Secondary Perio')
  os.makedirs(path)
  path=os.path.join(parent_dir,'Primary Endodontic Lesion')
  os.makedirs(path)
  path=os.path.join(parent_dir,'Primary Perio with Secondary Endo')
  os.makedirs(path)
  path=os.path.join(parent_dir,'Primary Periodontal Lesion')
  os.makedirs(path)
  path=os.path.join(parent_dir,'True Combined Lesions')
  os.makedirs(path)



  ### END CODE HERE


try:
  create_train_val_dirs(root_path=root_dir)
except FileExistsError:
  print("You should not be seeing this since the upper directory is removed beforehand")

In [7]:
for rootdir, dirs, files in os.walk(root_dir):
    for subdir in dirs:
        print(os.path.join(rootdir, subdir))

/content/Xrays/validation
/content/Xrays/training
/content/Xrays/validation/Primary Endo with Secondary Perio
/content/Xrays/validation/True Combined Lesions
/content/Xrays/validation/Primary Periodontal Lesion
/content/Xrays/validation/Primary Perio with Secondary Endo
/content/Xrays/validation/Primary Endodontic Lesion
/content/Xrays/training/Primary Endo with Secondary Perio
/content/Xrays/training/True Combined Lesions
/content/Xrays/training/Primary Periodontal Lesion
/content/Xrays/training/Primary Perio with Secondary Endo
/content/Xrays/training/Primary Endodontic Lesion


In [8]:
# split_data
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):
  """
  Splits the data into train and test sets

  Args:
    SOURCE_DIR (string): directory path containing the images
    TRAINING_DIR (string): directory path to be used for training
    VALIDATION_DIR (string): directory path to be used for validation
    SPLIT_SIZE (float): proportion of the dataset to be used for training

  Returns:
    None
  """

  shuffle_list=random.sample(os.listdir(SOURCE_DIR),len(os.listdir(SOURCE_DIR)))

  training_number=int(len(shuffle_list)*SPLIT_SIZE)

  i=0
  target=TRAINING_DIR

  for item in shuffle_list:
    item_source=os.path.join(SOURCE_DIR,item)
    if os.path.getsize(item_source)==0:
      print(f'{item}is zero length, so ignoring.')
    else:
      copyfile(item_source,os.path.join(target,item))
      i+=1

    if i==training_number:
      target=VALIDATION_DIR


In [9]:


# Define paths
Pr_En_wi_Se_Pe_SOURCE_DIR = "/content/Periapical_Xrays/Primary Endo with Secondary Perio"
Pr_En_Le_SOURCE_DIR = "/content/Periapical_Xrays/Primary Endodontic Lesion"
Pr_Pe_wi_En_SOURCE_DIR="/content/Periapical_Xrays/Primary Perio with Secondary Endo"
Pr_Pe_Le_SOURCE_DIR="/content/Periapical_Xrays/Primary Periodontal Lesion"
Tr_Co_Le_SOURCE_DIR="/content/Periapical_Xrays/True Combined Lesions"

TRAINING_DIR = "/content/Xrays/training"
VALIDATION_DIR = "/content/Xrays/validation"

TRAINING_Pr_En_wi_Se_Pe_DIR = os.path.join(TRAINING_DIR, "Primary Endo with Secondary Perio/")
TRAINING_Pr_En_Le_DIR=os.path.join(TRAINING_DIR,"Primary Endodontic Lesion/")
TRAINING_Pr_Pe_wi_En_DIR=os.path.join(TRAINING_DIR,"Primary Perio with Secondary Endo/")
TRAINING_Pr_Pe_Le_DIR=os.path.join(TRAINING_DIR,"Primary Periodontal Lesion/")
TRAINING_Tr_Co_Le_DIR=os.path.join(TRAINING_DIR,"True Combined Lesions/")


VALIDATION_Pr_En_wi_Se_Pe_DIR = os.path.join(VALIDATION_DIR, "Primary Endo with Secondary Perio/")
VALIDATION_Pr_En_Le_DIR=os.path.join(VALIDATION_DIR,"Primary Endodontic Lesion/")
VALIDATION_Pr_Pe_wi_En_DIR=os.path.join(VALIDATION_DIR,"Primary Perio with Secondary Endo/")
VALIDATION_Pr_Pe_Le_DIR=os.path.join(VALIDATION_DIR,"Primary Periodontal Lesion/")
VALIDATION_Tr_Co_Le_DIR=os.path.join(VALIDATION_DIR,"True Combined Lesions/")


# Define proportion of images used for training
split_size = .85

# Run the function
# NOTE: Messages about zero length images should be printed out
split_data(Pr_En_wi_Se_Pe_SOURCE_DIR, TRAINING_Pr_En_wi_Se_Pe_DIR,VALIDATION_Pr_En_wi_Se_Pe_DIR, split_size)
split_data(Pr_En_Le_SOURCE_DIR, TRAINING_Pr_En_Le_DIR,VALIDATION_Pr_En_Le_DIR, split_size)
split_data(Pr_Pe_wi_En_SOURCE_DIR,TRAINING_Pr_Pe_wi_En_DIR,VALIDATION_Pr_Pe_wi_En_DIR,split_size)
split_data(Pr_Pe_Le_SOURCE_DIR,TRAINING_Pr_Pe_Le_DIR,VALIDATION_Pr_Pe_Le_DIR,split_size)
split_data(Tr_Co_Le_SOURCE_DIR,TRAINING_Tr_Co_Le_DIR,VALIDATION_Tr_Co_Le_DIR,split_size)

# Check that the number of images matches the expected output

# Your function should perform copies rather than moving images so original directories should contain unchanged images
print(f"\n\nOriginal Primary Endo with Secondary Perio directory has {len(os.listdir(Pr_En_wi_Se_Pe_SOURCE_DIR))} images")
print(f"Original Primary Endodontic Lesion directory has {len(os.listdir(Pr_En_Le_SOURCE_DIR))} images\n")
print(f"Orignal Primary Perio with Secondary Endo has {len(os.listdir(Pr_Pe_wi_En_SOURCE_DIR))} images\n")
print(f"Orignal Primary Periodontal Lesion has {len(os.listdir(Pr_Pe_Le_SOURCE_DIR))} images\n")
print(f"Orignal True Combined Lesions has {len(os.listdir(Tr_Co_Le_SOURCE_DIR))} images\n")

# Training and validation splits
print(f"There are {len(os.listdir(TRAINING_Pr_En_wi_Se_Pe_DIR))} images of Primary Endo with Secondary Perio directory for training")
print(f"There are {len(os.listdir(TRAINING_Pr_En_Le_DIR))} images of Primary Endodontic Lesion directory for training")
print(f"There are {len(os.listdir(TRAINING_Pr_Pe_wi_En_DIR))} images of Primary Perio with Secondary Endo directory for training")
print(f"There are {len(os.listdir(TRAINING_Pr_Pe_Le_DIR))} images of Primary Periodontal Lesion directory for training")
print(f"There are {len(os.listdir(TRAINING_Tr_Co_Le_DIR))} images of True Combined Lesions directory for training\n")

print(f"There are {len(os.listdir(VALIDATION_Pr_En_wi_Se_Pe_DIR))} images of Primary Endo with Secondary Perio for validation")
print(f"There are {len(os.listdir(VALIDATION_Pr_En_Le_DIR))} images of Primary Endodontic Lesion for validation")
print(f"There are {len(os.listdir(VALIDATION_Pr_Pe_wi_En_DIR))} images of  Primary Perio with Secondary Endo for validation")
print(f"There are {len(os.listdir(VALIDATION_Pr_Pe_Le_DIR))} images of  Primary Periodontal Lesion for validaton")
print(f"There are {len(os.listdir(VALIDATION_Tr_Co_Le_DIR))} images of  True Combined Lesions for validation")



Original Primary Endo with Secondary Perio directory has 122 images
Original Primary Endodontic Lesion directory has 124 images

Orignal Primary Perio with Secondary Endo has 39 images

Orignal Primary Periodontal Lesion has 118 images

Orignal True Combined Lesions has 131 images

There are 103 images of Primary Endo with Secondary Perio directory for training
There are 105 images of Primary Endodontic Lesion directory for training
There are 33 images of Primary Perio with Secondary Endo directory for training
There are 100 images of Primary Periodontal Lesion directory for training
There are 111 images of True Combined Lesions directory for training

There are 19 images of Primary Endo with Secondary Perio for validation
There are 19 images of Primary Endodontic Lesion for validation
There are 6 images of  Primary Perio with Secondary Endo for validation
There are 18 images of  Primary Periodontal Lesion for validaton
There are 20 images of  True Combined Lesions for validation


In [11]:
# Read input images and assign labels based on folder names
print(os.listdir("Xrays/"))

['validation', 'training']


In [10]:
SIZE = 256  #Resize images

In [11]:
#Capture training data and labels into respective lists
train_images = []
train_labels = []

In [12]:
for directory_path in glob.glob("/content/Xrays/training/*"):
    train_label = directory_path.split("/")[-1]
    print(train_label)
    for img_path in glob.glob(os.path.join(directory_path, "*.JPG")):
      print(img_path)
      img = cv2.imread(img_path, cv2.IMREAD_COLOR)
      img = cv2.resize(img, (SIZE, SIZE))
      img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
      train_images.append(img)
      train_labels.append(train_label)

Primary Endo with Secondary Perio
/content/Xrays/training/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (84).JPG
/content/Xrays/training/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (89).JPG
/content/Xrays/training/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (113).JPG
/content/Xrays/training/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (26).JPG
/content/Xrays/training/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (38).JPG
/content/Xrays/training/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (111).JPG
/content/Xrays/training/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (88).JPG
/content/Xrays/training/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (69).JPG
/content/Xrays/training/Primary Endo with Secondary Perio/Primary Endo with Secondary Perio (116).JPG
/content/Xrays/training/Primary Endo with Secondary Pe

In [35]:
type(train_images)

list

In [13]:
#Convert lists to arrays
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [14]:
train_labels.shape

(452,)

In [15]:
train_images.shape

(452, 256, 256, 3)

In [16]:
# Capture test/validation data and labels into respective lists

test_images = []
test_labels = []
for directory_path in glob.glob("/content/Xrays/validation/*"):
    label = directory_path.split("/")[-1]
    print(label)

    for img_path in glob.glob(os.path.join(directory_path, "*.JPG")):

        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(label)

Primary Endo with Secondary Perio
True Combined Lesions
Primary Periodontal Lesion
Primary Perio with Secondary Endo
Primary Endodontic Lesion


In [17]:
#Convert lists to arrays
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [18]:
test_images.shape

(82, 256, 256, 3)

In [19]:
test_labels.shape

(82,)

In [20]:
test_labels

array(['Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio',
       'Primary Endo with Secondary Perio', 'True Combined Lesions',
       'True Combined Lesions', 'True Combined Lesions',
       'True Combined Lesions', 'True Combined Lesions',
       'True Combined Les

In [21]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

In [22]:
test_labels_encoded.shape

(82,)

In [23]:
# Normalize pixel values to between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

In [24]:
#One hot encode y values for neural network.
from keras.utils import to_categorical
y_train_one_hot = to_categorical(train_labels_encoded)
y_test_one_hot = to_categorical(test_labels_encoded)

In [67]:
y_test_one_hot.shape

(82, 5)

In [25]:
#Load model wothout classifier/fully connected layers
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

58889256/58889256 [==============================] - 0s 0us/step


In [26]:
#Make loaded layers as non-trainable. This is important as we want to work with pre-trained weights
for layer in VGG_model.layers:
	layer.trainable = False

In [27]:
VGG_model.summary()  #Trainable parameters will be 0

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 256, 256, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 256, 256, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 128, 128, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 128, 128, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 128, 128, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 64, 64, 128)       0     

In [28]:
#Now, let us use features from convolutional network for RF
feature_extractor=VGG_model.predict(train_images)

15/15 [==============================] - 377s 25s/step


In [29]:
feature_extractor.shape

(452, 8, 8, 512)

In [30]:
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

In [31]:
features.shape

(452, 32768)

In [32]:
X_for_SVM = features #This is our X input to SV

In [42]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [43]:
# Create an SVM classifier with the OvR strategy
SVM_Classifier = SVC(kernel="linear")

In [44]:
# Training the model on the training data and labels

SVM_Classifier.fit(X_for_SVM , train_labels_encoded)#For sklearn no one hot encoding

SVC(kernel='linear')

In [45]:
#Send test data through same feature extractor process
X_test_feature = VGG_model.predict(test_images)
X_test_features = X_test_feature.reshape(X_test_feature.shape[0], -1)

3/3 [==============================] - 68s 22s/step


In [46]:
# Using the model to predict the labels of the test data
prediction_SVM = SVM_Classifier.predict(X_test_features)



In [47]:
#Inverse le transform to get original label back.
prediction_SVM = le.inverse_transform(prediction_SVM)

In [48]:
prediction_SVM[1]

'Primary Endo with Secondary Perio'

In [49]:
test_labels[1]

'Primary Endo with Secondary Perio'

In [50]:
# Calculate accuracy
accuracy = accuracy_score(test_labels, prediction_SVM)
print("Accuracy:", accuracy)

Accuracy: 0.4268292682926829
